In [ ]:
# create a file with all requirements
%%writefile requirements.txt
openai
tiktoken
pandas
transformers
plotly
matplotlib
torch
torchvision
scipy

Overwriting requirements.txt


In [ ]:
# install all requirements
%pip install -r requirements.txt

  Using cached openai-0.27.8-py3-none-any.whl (73 kB)
  Using cached tiktoken-0.4.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
  Using cached transformers-4.31.0-py3-none-any.whl (7.4 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.8 MB/s eta 0:00:00


In [ ]:
# openai api key
import openai
openai.api_key = 'REDACTED'

import pandas as pd
import tiktoken
from openai.embeddings_utils import get_embedding

In [ ]:
# embedding model parameters
embedding_model = "text-embedding-ada-002"
embedding_encoding = "cl100k_base"  # this the encoding for text-embedding-ada-002
max_tokens = 8000  # the maximum for text-embedding-ada-002 is 8191

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

# load product data from csv file
product_df = pd.read_csv('/content/drive/MyDrive/Job Search 2023/coding challenge/instalily/data/product_df_with_metadata.csv')

product_df['type_'] = product_df['type']

product_df['size'] = product_df['size'].astype(str).str.strip()
product_df['firmness'] = product_df['firmness'].astype(str).str.strip()
product_df['type_'] = product_df['type_'].astype(str).str.strip()
product_df['material'] = product_df['material'].astype(str).str.strip()

# combine all cols into single string to be used for embedding
product_df["combined"] = (
    "link: " + product_df.link.str.strip() +
    "; product name: " + product_df.product_name.str.strip() +
    "; product desc: " + product_df.product_desc.str.strip() +
    "; product type: " + product_df.product_type.str.strip() +
    "; product rating (out of 5): " + product_df.product_rating.astype(str) +
    "; num_reviews: " + product_df.num_reviews.astype(str) +
    "; size: " + product_df['size'].astype(str) +
    "; firmness: " + product_df.firmness.astype(str) +
    "; price: " + product_df.price.astype(str) +
    "; type: " + product_df.type_.astype(str) +
    "; material: " + product_df.material.astype(str) +
    "; metadata: " + product_df.metadata.str.strip()
)
product_df.head(2)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,link,product_name,product_desc,product_type,product_rating,num_reviews,size,firmness,price,multi_price,type,material,metadata,type_,combined
0,https://www.saatva.com/mattresses/saatva-classic,Saatva Classic Mattress,America's best-selling online luxury innerspring,mattresses,4.8,3000,king,plush+soft,2195.0,False,NaN,nan,{'product_info': 'Bed Bonanza\nBIG Bedroom Bon...,nan,link: https://www.saatva.com/mattresses/saatva...
1,https://www.saatva.com/mattresses/saatva-classic,Saatva Classic Mattress,America's best-selling online luxury innerspring,mattresses,4.8,3000,king,medium+firm,2195.0,False,NaN,nan,{'product_info': 'Bed Bonanza\nBIG Bedroom Bon...,nan,link: https://www.saatva.com/mattresses/saatva...


In [ ]:
len(product_df)

584

In [ ]:
# get_encoding takes text as input and returns its corresponding token encoding
encoding = tiktoken.get_encoding(embedding_encoding)

# create a new column n_tokens
product_df["n_tokens"] = product_df.combined.apply(lambda x: len(encoding.encode(x)))

# omit reviews that are too long to embed
df = product_df[product_df.n_tokens <= max_tokens]
# print length of dataframe to make sure it isn't too long
len(df)

584

In [ ]:
df["embedding"] = df.combined.apply(lambda x: get_embedding(x, engine=embedding_model))

In [ ]:
df.to_csv('/content/drive/MyDrive/Job Search 2023/coding challenge/instalily/data/product_encoding_v1.csv', index=False)